In [1]:
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [2]:
import torch

# DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch.set_default_device(DEVICE)

In [3]:
!pip install datasets==2.14.7

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 3.6 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.3.0
    Uninstalling fsspec-2024.3.0:
      Successfully uninstalled fsspec-2024.3.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling multiprocess-0.70.16:
      Successfully uninstalled multiprocess-0.70.16
  Attempting uninstall: datasets
 

In [4]:
import datasets

train_data = datasets.load_dataset('wmt16','de-en',split='train[:50000]')
val_data = datasets.load_dataset('wmt16',"de-en", split="validation")
test_data = datasets.load_dataset('wmt16','de-en', split="test")

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

In [5]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 934.5 kB/s eta 0:00:000:00:010m


In [6]:
!pip install transformers --upgrade nltk --upgrade bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 30.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 40.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.


In [7]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import Dataset
import evaluate
import matplotlib.pyplot as plt

2024-03-31 16:58:35.221111: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-31 16:58:35.221293: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-31 16:58:35.402779: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
bleu_metric = evaluate.load('bleu')
meteor_metric = evaluate.load('meteor')
bertscore_metric = evaluate.load('bertscore')

# Load pre-trained T5 model and tokenizer
checkpoint = "t5-small"
tokenizer_2c = AutoTokenizer.from_pretrained(checkpoint)
model_2c = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [9]:
def tokenize_function(example):
    example['de'] = tokenizer_2c(example['de'], padding=True, truncation=True, return_tensors="pt")
    example['en'] = tokenizer_2c(example['en'], padding=True, truncation=True, return_tensors="pt")
    return example

In [10]:
model_input = Dataset.from_list(train_data['translation']).map(tokenize_function)
model_val = Dataset.from_list(val_data['translation']).map(tokenize_function)
model_test = Dataset.from_list(test_data['translation']).map(tokenize_function)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2169 [00:00<?, ? examples/s]

Map:   0%|          | 0/2999 [00:00<?, ? examples/s]

In [11]:
# Define translation function
def translate_2C(model,tokenizer,text:str):
    input_text = "translate German to English: " + text
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    outputs = model.generate(input_ids)
    en_translation = tokenizer.decode(outputs[0])

    return en_translation

In [12]:
# Define evaluation function
def evaluate(model,tokenizer,data):
    translations = []
    references = []
    for item in data:
        de_text = item['translation']['de']
        en_translation = translate_2C(model,tokenizer,de_text)
        translations.append(en_translation)
        references.append(item['translation']['en'])

    bleu = bleu_metric.compute(predictions=translations, references=references)
    meteor = meteor_metric.compute(predictions=translations, references=references)
    bertscore = bertscore_metric.compute(predictions=translations, references=references, model_type="distilbert-base-uncased")

    return bleu, meteor, bertscore

In [14]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    overwrite_output_dir=True,
    save_total_limit=3,
#     fp16=True
)

# Define trainer
trainer = Seq2SeqTrainer(
    model=model_2c,
    tokenizer=tokenizer_2c,
    args=training_args,
    train_dataset=model_input,
    eval_dataset=model_val,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model, max_length=256),
    compute_metrics=evaluate
)

# Train the model
trainer.train()
trainer.save_model('fine_tuned_t5-small')

train_loss = trainer.state.log_history["loss"]
val_loss = trainer.state.log_history["eval_loss"]

# Plot the loss curves
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


IndexError: Invalid key: 37542 is out of bounds for size 0

In [15]:
# Evaluate on validation data after training
val_bleu, val_meteor, val_bertscore = evaluate(model_2c,tokenizer_2c,val_data)
print("Validation Metrics:")
print(f"BLEU-1: {val_bleu['precisions'][0]}")
print(f"BLEU-2: {val_bleu['precisions'][1]}")
print(f"BLEU-3: {val_bleu['precisions'][2]}")
print(f"BLEU-4: {val_bleu['precisions'][3]}")
print(f"Meteor Score: {val_meteor['score']}")
print(f"Validation BERTScore: {val_bertscore['score']}")

# Evaluate on test data
test_bleu, test_meteor, test_bertscore = evaluate(model_2c,tokenizer_2c,test_data)
print("Test Metrics:")
print(f"BLEU-1: {test_bleu['precisions'][0]}")
print(f"BLEU-2: {test_bleu['precisions'][1]}")
print(f"BLEU-3: {test_bleu['precisions'][2]}")
print(f"BLEU-4: {test_bleu['precisions'][3]}")
print(f"Test METEOR: {test_meteor['meteor']}")
print(f"Test BERTScore: {sum(test_bertscore['precision'])/len(test_bertscore['precision'])}")

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
def translate_sentence(sentence:str):
    print("Original Sentence: ", sentence)
    print(f"2A: {translate_2A(translator, tokenizer_de, tokenizer_en, sentence)}")
    print(f"2B: {translate_2B(model_2b, tokenizer, sentence)}")
    print(f"2C: {translate_2C(sentence)}")